In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os,sys
opj = os.path.join
from copy import deepcopy
from tqdm import tqdm

from ex_mnist import p
from dset import get_dataloader, load_pretrained_model

sys.path.append('../../src/models')
from models import CNN, FFN

In [37]:
# load data and model
train_loader, test_loader = get_dataloader(p.data_path,
                                           batch_size=p.batch_size)

cnn, ffn = load_pretrained_model(p.model_path)    

In [38]:
# check prediction
m = len(test_loader.dataset)
batch_size = test_loader.batch_size

y_pred_cnn = np.zeros(m)
y_pred_ffn = np.zeros(m)
y_true = np.zeros(m)
with torch.no_grad():
    for batch_idx, (data, y) in tqdm(enumerate(test_loader, 0), total=int(np.ceil(m / batch_size))):
        data = data.to(device)
        # cnn prediction
        outputs_cnn = cnn(data)
        _, y_pred = torch.max(outputs_cnn.data, 1)
        y_pred_cnn[batch_idx*batch_size:(batch_idx+1)*batch_size] = y_pred.cpu().numpy()
        
        # ffn prediction
        outputs_ffn = ffn(data)
        _, y_pred = torch.max(outputs_ffn.data, 1)
        y_pred_ffn[batch_idx*batch_size:(batch_idx+1)*batch_size] = y_pred.cpu().numpy() 
        
        # labels
        y_true[batch_idx*batch_size:(batch_idx+1)*batch_size] = y.numpy()
        
print("CNN accuracy {:.5f}% FFN accuracy {:.5f}%".format((y_true == y_pred_cnn).sum()/m*100,
                                                       (y_true == y_pred_ffn).sum()/m*100))

100%|██████████| 100/100 [00:03<00:00, 33.09it/s]

CNN accuracy 99.09000% FFN accuracy 98.28000%
